In [428]:
import os
import csv
import sys
import re
import pandas as pd

from sklearn.externals import joblib
from surprise import Dataset
from surprise import Reader
from surprise import SVD
from Movielens import Movielens

In [441]:
model = joblib.load("./SVDtuned.pkl")
ml = Movielens()
MovieMatrix, RatingMatrix = ml.loadDataSet()
MovieMatrix.to_pickle("movie_data.pkl")

In [442]:
user_feature = model.pu
movie_feature = model.qi

In [444]:
import numpy as np
import random
from random import randint
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam, SGD

class ENV:
    def __init__(self, User_Feature, Movie_Feature, User_Matrix, Movie_Matrix, model):
        self.User_Feature = User_Feature
        self.Movie_Feature = Movie_Feature
        self.User_Matrix = User_Matrix
        self.Movie_Matrix = Movie_Matrix
        self.UserMovie = self.combine_matrix().fillna(0)
        self.state_size = self.UserMovie.shape[1] #user feature
        self.action_size = self.UserMovie.shape[1] #num of movie (actions)
        self.SVDmodel = model
    
    def combine_matrix(self):
        merged=pd.merge(self.User_Matrix,self.Movie_Matrix,left_on='movieId',right_on="movieId")
        return pd.pivot_table(merged, values='rating',
                                   index=['userId'], columns=['movieId'])
        
    def random_select_user(self):
        num = random.randint(1,610) #random pick from 610 user
        self.selectedUser = self.UserMovie.loc[[num]].to_numpy()
        self.selectedUser_num = num
        return self.selectedUser
    
#     def prediction_model(self):
        
    def current_state(self):
        return self.selectedUser
    
    def step(self, action): #given an action to return reward + next_state
        #action in an item recommended by the agent(recommender)
        #real is for real user
#         randomRating = random.randint(0,5)
#         reward = -1 if randomRating == 0 else randomRating
        
#         if(randomRating == 0):
#             reward = -1 #we give a reward of 0 if user not rate (i.e. = 0)
#         else:
#             reward = randomRating #reward just equal to the rating
        #action is in raw id
        to_real_movieID = int(env.Movie_Matrix.iloc[action]['movieId'])
        
        prediction = self.SVDmodel.predict(self.selectedUser_num, to_real_movieID)
        estimatedRating = prediction[3]
        
        reward = estimatedRating
        self.selectedUser[0,action] = reward #update the user's state(history)
        next_state = self.selectedUser
        
        return next_state, reward     

In [445]:
env = ENV(user_feature, movie_feature, RatingMatrix, MovieMatrix, SVDmodel)

In [375]:
env.random_select_user()

array([[3.5, 0. , 0. , ..., 0. , 0. , 0. ]])

In [372]:
env.state_size

9724

In [373]:
env.action_size

9724

In [376]:
env.selectedUser

array([[3.5, 0. , 0. , ..., 0. , 0. , 0. ]])

In [446]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000) #use it for storing history
        self.GAMMA = 0.9
        self.EPSILON = 1.0 #let user to explore at first, it will decay
        self.EPSILON_DECAY = 0.995
        self.EPSILON_MIN = 0.01
        self.learning_rate = 0.001 #for model
        self.model = self._build_model()
    
    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=SGD(lr=self.learning_rate)
                      , metrics=['accuracy'])
        
        return model
    
    def remember_memory(self, state, action, reward, next_state):
        self.memory.append((state, action, reward, next_state))
  
    def make_act(self, state):
        if np.random.rand() <= 0.3: #exploration
            restart = True
            while restart:
                restart = False
                action = random.randrange(self.action_size)
                for i in range(len(self.memory)):
                    if self.memory[i][1] == action:
                        restart = True
                        break             
            return action
        action_values = self.model.predict(state)
        return np.argmax(action_values[0]) #exploitation
    
    def experience_replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state in minibatch:
            target = reward + self.GAMMA * np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
#         if self.EPSILON > self.EPSILON_MIN:
#             self.EPSILON *= self.EPSILON_DECAY


In [447]:
agent = DQNAgent(env.state_size, env.action_size) #intialise agent
batch_size = 32

In [448]:
EPISODE = 200
TIME_STEPS = 40
for e in range(500):
    state = env.random_select_user()
    #print(state.shape)
    for time in range(TIME_STEPS):
        action = agent.make_act(state)
        print("action: {}".format(action))
        next_state, reward = env.step(action)
        print("action: {}, reward: {}, e:{:.2}".format(action, reward, agent.EPSILON))
        agent.remember_memory(state,action,reward,next_state)
        state = next_state
    if len(agent.memory) > batch_size:
        agent.experience_replay(batch_size)

action: 7510
action: 7510, reward: 3.843670213355653, e:1.0
action: 7510
action: 7510, reward: 3.843670213355653, e:1.0
action: 7510
action: 7510, reward: 3.843670213355653, e:1.0
action: 7510
action: 7510, reward: 3.843670213355653, e:1.0
action: 7510
action: 7510, reward: 3.843670213355653, e:1.0
action: 7510
action: 7510, reward: 3.843670213355653, e:1.0
action: 7510
action: 7510, reward: 3.843670213355653, e:1.0
action: 7510
action: 7510, reward: 3.843670213355653, e:1.0
action: 4714
action: 4714, reward: 3.5108978073442048, e:1.0
action: 7510
action: 7510, reward: 3.843670213355653, e:1.0
action: 7359
action: 7359, reward: 3.4982543405494955, e:1.0
action: 7510
action: 7510, reward: 3.843670213355653, e:1.0
action: 7076
action: 7076, reward: 3.5902041850540725, e:1.0
action: 7510
action: 7510, reward: 3.843670213355653, e:1.0
action: 9252
action: 9252, reward: 3.590603534019707, e:1.0
action: 7626
action: 7626, reward: 3.2190783193789674, e:1.0
action: 8512
action: 8512, reward: 3

action: 2999
action: 2999, reward: 2.836543199598817, e:1.0
action: 6263
action: 6263, reward: 3.48341347359965, e:1.0
action: 7510
action: 7510, reward: 3.843670213355653, e:1.0
action: 7510
action: 7510, reward: 3.843670213355653, e:1.0
action: 7510
action: 7510, reward: 3.843670213355653, e:1.0
action: 7510
action: 7510, reward: 3.843670213355653, e:1.0
action: 8136
action: 8136, reward: 3.5660543656563677, e:1.0
action: 3243
action: 3243, reward: 3.385484866567039, e:1.0
action: 4929
action: 4929, reward: 3.434076188318906, e:1.0
action: 2306
action: 2306, reward: 3.6805802714535467, e:1.0
action: 953
action: 953, reward: 3.7073566089782983, e:1.0
action: 4929
action: 4929, reward: 3.434076188318906, e:1.0
action: 3853
action: 3853, reward: 3.6568136469492623, e:1.0
action: 2670
action: 2670, reward: 3.015812551131464, e:1.0
action: 2670
action: 2670, reward: 3.015812551131464, e:1.0
action: 2670
action: 2670, reward: 3.015812551131464, e:1.0
action: 2670
action: 2670, reward: 3.01

action: 7456
action: 7456, reward: 3.7995452959129983, e:1.0
action: 3807
action: 3807, reward: 3.7272701412705826, e:1.0
action: 3807
action: 3807, reward: 3.7272701412705826, e:1.0
action: 2813
action: 2813, reward: 3.522498288628954, e:1.0
action: 3824
action: 3824, reward: 3.313872104293282, e:1.0
action: 3807
action: 3807, reward: 3.7272701412705826, e:1.0
action: 3807
action: 3807, reward: 3.7272701412705826, e:1.0
action: 3807
action: 3807, reward: 3.7272701412705826, e:1.0
action: 4318
action: 4318, reward: 3.299728592150476, e:1.0
action: 3807
action: 3807, reward: 3.7272701412705826, e:1.0
action: 6114
action: 6114, reward: 3.508640176013213, e:1.0
action: 3807
action: 3807, reward: 3.7272701412705826, e:1.0
action: 3807
action: 3807, reward: 3.7272701412705826, e:1.0
action: 989
action: 989, reward: 3.329716455189647, e:1.0
action: 4186
action: 4186, reward: 3.5995865054200253, e:1.0
action: 3807
action: 3807, reward: 3.7272701412705826, e:1.0
action: 3807
action: 3807, rewa

action: 7531
action: 7531, reward: 3.607388201449843, e:1.0
action: 7531
action: 7531, reward: 3.607388201449843, e:1.0
action: 7531
action: 7531, reward: 3.607388201449843, e:1.0
action: 7531
action: 7531, reward: 3.607388201449843, e:1.0
action: 7531
action: 7531, reward: 3.607388201449843, e:1.0
action: 7531
action: 7531, reward: 3.607388201449843, e:1.0
action: 3563
action: 3563, reward: 3.4202104299091944, e:1.0
action: 8408
action: 8408, reward: 3.623301745467128, e:1.0
action: 951
action: 951, reward: 4.156313566399177, e:1.0
action: 9225
action: 9225, reward: 3.5008761961457227, e:1.0
action: 5187
action: 5187, reward: 3.6056403168443785, e:1.0
action: 7531
action: 7531, reward: 3.607388201449843, e:1.0
action: 7531
action: 7531, reward: 3.607388201449843, e:1.0
action: 1634
action: 1634, reward: 3.514445735115456, e:1.0
action: 7531
action: 7531, reward: 3.607388201449843, e:1.0
action: 7531
action: 7531, reward: 3.607388201449843, e:1.0
action: 7531
action: 7531, reward: 3.60

action: 6791
action: 6791, reward: 3.1487362734306203, e:1.0
action: 5147
action: 5147, reward: 3.5864004637412883, e:1.0
action: 6791
action: 6791, reward: 3.1487362734306203, e:1.0
action: 3254
action: 3254, reward: 3.3211783814168676, e:1.0
action: 6791
action: 6791, reward: 3.1487362734306203, e:1.0
action: 1889
action: 1889, reward: 3.683620536082818, e:1.0
action: 6791
action: 6791, reward: 3.1487362734306203, e:1.0
action: 6791
action: 6791, reward: 3.1487362734306203, e:1.0
action: 6791
action: 6791, reward: 3.1487362734306203, e:1.0
action: 6791
action: 6791, reward: 3.1487362734306203, e:1.0
action: 5765
action: 5765, reward: 3.578549423242836, e:1.0
action: 6791
action: 6791, reward: 3.1487362734306203, e:1.0
action: 6791
action: 6791, reward: 3.1487362734306203, e:1.0
action: 6791
action: 6791, reward: 3.1487362734306203, e:1.0
action: 6791
action: 6791, reward: 3.1487362734306203, e:1.0
action: 9658
action: 9658, reward: 3.5660587813465945, e:1.0
action: 6791
action: 6791,

...

action: 7174
action: 7174, reward: 3.7049911002165112, e:1.0
action: 8691
action: 8691, reward: 3.770810242371262, e:1.0
action: 1570
action: 1570, reward: 3.5646244814636385, e:1.0
action: 8691
action: 8691, reward: 3.770810242371262, e:1.0
action: 8691
action: 8691, reward: 3.770810242371262, e:1.0
action: 8691
action: 8691, reward: 3.770810242371262, e:1.0
action: 8691
action: 8691, reward: 3.770810242371262, e:1.0
action: 8691
action: 8691, reward: 3.770810242371262, e:1.0
action: 163
action: 163, reward: 2.542368921956787, e:1.0
action: 7101
action: 7101, reward: 3.4101947267611226, e:1.0
action: 8691
action: 8691, reward: 3.770810242371262, e:1.0
action: 8691
action: 8691, reward: 3.770810242371262, e:1.0
action: 2859
action: 2859, reward: 2.975057428143523, e:1.0
action: 8691
action: 8691, reward: 3.770810242371262, e:1.0
action: 4043
action: 4043, reward: 3.5791605462032434, e:1.0
action: 6437
action: 6437, reward: 3.6348278903620437, e:1.0
action: 8691
action: 8691, reward: 3.

action: 3485
action: 3485, reward: 3.5325611423372867, e:1.0
action: 2198
action: 2198, reward: 3.653810577279868, e:1.0
action: 486
action: 486, reward: 3.7843083716550936, e:1.0
action: 2803
action: 2803, reward: 3.4837122455360277, e:1.0
action: 4676
action: 4676, reward: 3.599290755081097, e:1.0
action: 486
action: 486, reward: 3.7843083716550936, e:1.0
action: 486
action: 486, reward: 3.7843083716550936, e:1.0
action: 6421
action: 6421, reward: 3.482087493738455, e:1.0
action: 8673
action: 8673, reward: 3.5110949038020594, e:1.0
action: 486
action: 486, reward: 3.7843083716550936, e:1.0
action: 8149
action: 8149, reward: 3.4688711637866363, e:1.0
action: 8516
action: 8516, reward: 3.7147184011306065, e:1.0
action: 486
action: 486, reward: 3.7843083716550936, e:1.0
action: 486
action: 486, reward: 3.7843083716550936, e:1.0
action: 486
action: 486, reward: 3.7843083716550936, e:1.0
action: 486
action: 486, reward: 3.7843083716550936, e:1.0
action: 5834
action: 5834, reward: 3.419378

action: 8050
action: 8050, reward: 3.479721745451122, e:1.0
action: 2672
action: 2672, reward: 3.4995032103481045, e:1.0
action: 2430
action: 2430, reward: 3.6871672182041864, e:1.0
action: 2430
action: 2430, reward: 3.6871672182041864, e:1.0
action: 9682
action: 9682, reward: 3.5099530403014336, e:1.0
action: 2430
action: 2430, reward: 3.6871672182041864, e:1.0
action: 2430
action: 2430, reward: 3.6871672182041864, e:1.0
action: 2329
action: 2329, reward: 3.7272964505927972, e:1.0
action: 9097
action: 9097, reward: 3.2341898080344196, e:1.0
action: 6837
action: 6837, reward: 2.6410290075335974, e:1.0
action: 2430
action: 2430, reward: 3.6871672182041864, e:1.0
action: 2430
action: 2430, reward: 3.6871672182041864, e:1.0
action: 2430
action: 2430, reward: 3.6871672182041864, e:1.0
action: 2430
action: 2430, reward: 3.6871672182041864, e:1.0
action: 2430
action: 2430, reward: 3.6871672182041864, e:1.0
action: 2430
action: 2430, reward: 3.6871672182041864, e:1.0
action: 2430
action: 2430

action: 999
action: 999, reward: 3.7901824648257736, e:1.0
action: 1219
action: 1219, reward: 3.4773800558754973, e:1.0
action: 3577
action: 3577, reward: 3.3905686775814394, e:1.0
action: 1219
action: 1219, reward: 3.4773800558754973, e:1.0
action: 1219
action: 1219, reward: 3.4773800558754973, e:1.0
action: 5331
action: 5331, reward: 3.3298848144303226, e:1.0
action: 1219
action: 1219, reward: 3.4773800558754973, e:1.0
action: 1219
action: 1219, reward: 3.4773800558754973, e:1.0
action: 1219
action: 1219, reward: 3.4773800558754973, e:1.0
action: 1219
action: 1219, reward: 3.4773800558754973, e:1.0
action: 4387
action: 4387, reward: 3.464043004327347, e:1.0
action: 1219
action: 1219, reward: 3.4773800558754973, e:1.0
action: 1219
action: 1219, reward: 3.4773800558754973, e:1.0
action: 1219
action: 1219, reward: 3.4773800558754973, e:1.0
action: 2876
action: 2876, reward: 3.5610726007727327, e:1.0
action: 1930
action: 1930, reward: 3.565898469871533, e:1.0
action: 2948
action: 2948, r

action: 3513
action: 3513, reward: 3.6126825136608276, e:1.0
action: 6199
action: 6199, reward: 3.696258816729257, e:1.0
action: 3513
action: 3513, reward: 3.6126825136608276, e:1.0
action: 3513
action: 3513, reward: 3.6126825136608276, e:1.0
action: 5029
action: 5029, reward: 3.5151897854043748, e:1.0
action: 3513
action: 3513, reward: 3.6126825136608276, e:1.0
action: 3513
action: 3513, reward: 3.6126825136608276, e:1.0
action: 4492
action: 4492, reward: 3.5608877050111327, e:1.0
action: 5775
action: 5775, reward: 3.8112160272311773, e:1.0
action: 4210
action: 4210, reward: 3.7712002078761664, e:1.0
action: 3513
action: 3513, reward: 3.6126825136608276, e:1.0
action: 7554
action: 7554, reward: 3.9056821804572204, e:1.0
action: 2239
action: 2239, reward: 3.383729101400716, e:1.0
action: 3513
action: 3513, reward: 3.6126825136608276, e:1.0
action: 3513
action: 3513, reward: 3.6126825136608276, e:1.0
action: 3513
action: 3513, reward: 3.6126825136608276, e:1.0
action: 3513
action: 3513,

In [449]:
b = agent.model.predict(env.selectedUser)
print(b[0])
b[0].argsort()[-10:][::-1]

[ 0.03008332  0.00090214  0.0321472  ... -0.01717252 -0.00748099
 -0.02209356]


array([4780, 6481, 7510, 4677, 1359, 8315, 8537, 8165, 9346, 2670])

In [450]:
env.random_select_user()


array([[0., 0., 0., ..., 0., 0., 0.]])

In [383]:
env.selectedUser

array([[4., 0., 0., ..., 0., 0., 0.]])

In [451]:
a = agent.model.predict(env.selectedUser)
a[0].argsort()[-10:][::-1]

array([7531,  935, 2617, 5747, 6837, 5584, 6481,  944, 9391,  469])

In [385]:
np.argmax(a[0])

9638

In [419]:
env.UserMovie.tail()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
606,2.5,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
609,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
610,5.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [417]:
env.UserMovie.shape

(610, 9724)

In [452]:
joblib.dump(agent.model, "DQNmodel.pkl")

['DQNmodel.pkl']